In [37]:
import sys
sys.path.append('..\.')
from utils.qSLP import qSLP
from utils.Utils_pad import padding
from utils.Utils import get_params
from utils.import_data import get_dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import COBYLA
from qiskit import Aer, QuantumCircuit

from sklearn.metrics import accuracy_score

seed = 42
max_iter = 150

## Set the specification for the model

In [28]:
# Change d to change the number of control quibt. Currenly supports d <= 3
d = 1

# Change pad to change the state preparation technique
pad = False

# Change to change the dataset
dataset = "iris01"


## Training of the model
We train the qSLP model starting from given starting points


In [29]:

if pad:
    model_name = f"pad_qSLP_{d}"
else:
    model_name = f"sdq_qSLP_{d}"
#function that returns the best parameters for a given model
starting_point = get_params(model_name, dataset, "starting_points", "../results/training/file_result.txt")


### Obtain the dataset
The dataset is processed through the use a PCA in order to use only two high descriptive features. When a padded model is choosen the two feature are further preprocessed.


In [30]:
X_train, X_test, Y_train, Y_test = get_dataset(dataset)
if pad:
    X_train = padding(X_train)
    X_test = padding(X_test)


[0.90539269 0.07445563]
98.0% of total variance is explained by 2 principal components


### Set the optimizer and the quantum instance


In [31]:
optimizer = COBYLA(maxiter=max_iter, tol=0.01, disp=False)
qinstance = QuantumInstance(Aer.get_backend('aer_simulator'),seed_simulator=seed,seed_transpiler=seed, shots=1024)
qinstance.backend.set_option("seed_simulator", seed)


### The model 
Build the model with the chosen parameters.


In [32]:
model = qSLP(d, pad, seed=seed)
model.set_weights(starting_point)

## Training


In [34]:
hist = model.train(X_train, Y_train, optimizer, qinstance)


NameError: name 'accuracy_score' is not defined

In [38]:
train_acc = accuracy_score(Y_train, model.predict(X_train, qinstance))
test_acc = accuracy_score(Y_test,model.predict(X_test, qinstance))


In [39]:
print(train_acc)
print(test_acc)

1.0
1.0


In [41]:
starting_point

[0.41572934,
 0.15460859,
 0.53619502,
 0.12353289,
 0.6298117,
 0.69948602,
 0.47958129]

In [42]:
model.get_weights()

array([ 0.01218804,  2.41593477, -0.19823727,  1.87791022,  2.86343112,
        0.53337317,  0.06913996])

In [49]:
get_params(model_name, dataset, "ending_points", "../results/training/file_result.txt")

[0.01218804,
 2.41593477,
 -0.19823727,
 1.87791022,
 2.86343112,
 0.53337317,
 0.06913996]